## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-15-21-51-49 +0000,missionlocal,Mayor Lurie takes credit for immigrant legal f...,https://missionlocal.org/2025/08/sf-mayor-luri...
1,2025-08-15-21-50-53 +0000,nypost,75 homeless camps in DC cleared by US Park Pol...,https://nypost.com/2025/08/15/us-news/75-homel...
2,2025-08-15-21-49-20 +0000,nypost,Oklahoma starts giving tests to teachers from ...,https://nypost.com/2025/08/15/us-news/oklahoma...
3,2025-08-15-21-49-01 +0000,nypost,White-powder envelope sent to NYC ICE office s...,https://nypost.com/2025/08/15/us-news/white-po...
4,2025-08-15-21-47-58 +0000,nyt,Live Updates: Trump and Putin Meet for Summit ...,https://www.nytimes.com/live/2025/08/15/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
17,trump,60
45,putin,31
49,ukraine,18
108,new,17
47,summit,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
104,2025-08-15-15-56-48 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...,182
134,2025-08-15-13-25-54 +0000,nypost,Putin’s foreign minister trolls Ukraine by wea...,https://nypost.com/2025/08/15/world-news/putin...,170
246,2025-08-15-01-00-00 +0000,wsj,"President Trump, by meeting Vladimir Putin on ...",https://www.wsj.com/politics/national-security...,169
215,2025-08-15-06-37-53 +0000,cbc,Trump and Putin in Alaska to discuss war in Uk...,https://www.cbc.ca/9.6867437?cmp=rss,153
4,2025-08-15-21-47-58 +0000,nyt,Live Updates: Trump and Putin Meet for Summit ...,https://www.nytimes.com/live/2025/08/15/world/...,141


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
104,182,2025-08-15-15-56-48 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...
24,56,2025-08-15-20-58-21 +0000,nypost,Trump reacts to Hillary Clinton saying she’d n...,https://nypost.com/2025/08/15/us-news/trump-re...
221,55,2025-08-15-04-27-14 +0000,nypost,AG Pam Bondi declares DC ‘will not remain a sa...,https://nypost.com/2025/08/15/us-news/ag-pam-b...
275,53,2025-08-14-22-26-00 +0000,wsj,California Democrats are planning their own re...,https://www.wsj.com/politics/policy/gavin-news...
188,42,2025-08-15-09-30-00 +0000,wsj,Tariffs haven’t caused inflation to surge as m...,https://www.wsj.com/economy/trade/trump-tariff...
133,39,2025-08-15-13-29-05 +0000,nypost,Explosion at Russian gunpowder factory kills 5...,https://nypost.com/2025/08/15/world-news/russi...
55,35,2025-08-15-19-20-08 +0000,nypost,High-ranking Hamas terrorist killed by IDF ami...,https://nypost.com/2025/08/15/world-news/hamas...
276,34,2025-08-14-22-22-35 +0000,nypost,The first-ever World Humanoid Robot Games kick...,https://nypost.com/2025/08/14/tech/the-first-e...
1,29,2025-08-15-21-50-53 +0000,nypost,75 homeless camps in DC cleared by US Park Pol...,https://nypost.com/2025/08/15/us-news/75-homel...
247,28,2025-08-15-01-00-00 +0000,wsj,The nine law firms that promised President Tru...,https://www.wsj.com/politics/policy/trump-pro-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
